In [16]:
import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from yellowbrick.cluster import KElbowVisualizer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import dendrogram, linkage

In [31]:
df_customers = pd.read_csv("https://dojo-git.s3.eu-west-3.amazonaws.com/cleaned/customers.csv", sep=";")
df_products = pd.read_csv("https://dojo-git.s3.eu-west-3.amazonaws.com/cleaned/products.csv", sep=";")
df_transactions = pd.read_csv("https://dojo-git.s3.eu-west-3.amazonaws.com/cleaned/Transactions.csv", sep=";")

In [32]:
df_customers

,client_id,sex,birth
0,c_4410,f,1967
1,c_7839,f,1975
2,c_1699,f,1984
3,c_5961,f,1962
4,c_5320,m,1943
...,...,...,...
8616,c_7920,m,1956
8617,c_7403,f,1970
8618,c_5119,m,1974
8619,c_5643,f,1968


In [34]:
df_products

,id_prod,price,categ
0,0_1421,19.99,0
1,0_1368,5.13,0
2,0_731,17.99,0
3,1_587,4.99,1
4,0_1507,3.99,0
...,...,...,...
3281,2_23,115.99,2
3282,0_146,17.14,0
3283,0_802,11.22,0
3284,1_140,38.56,1


In [35]:
df_transactions

,id_prod,date,session_id,client_id
0,0_1259,2021-03-01 00:01:07.843138,s_1,c_329
1,0_1390,2021-03-01 00:02:26.047414,s_2,c_664
2,0_1352,2021-03-01 00:02:38.311413,s_3,c_580
3,0_1458,2021-03-01 00:04:54.559692,s_4,c_7912
4,0_1358,2021-03-01 00:05:18.801198,s_5,c_2033
...,...,...,...,...
687529,1_508,2023-02-28 23:49:03.148402,s_348444,c_3573
687530,2_37,2023-02-28 23:51:29.318531,s_348445,c_50
687531,1_695,2023-02-28 23:53:18.929676,s_348446,c_488
687532,0_1547,2023-02-28 23:58:00.107815,s_348447,c_4848


In [36]:
df_merge1 = pd.merge(df_transactions, df_products, on='id_prod', how='left')

In [37]:
df_merge = pd.merge(df_merge1, df_customers, on='client_id', how='left')

In [38]:
df_merge


,id_prod,date,session_id,client_id,price,categ,sex,birth
0,0_1259,2021-03-01 00:01:07.843138,s_1,c_329,11.99,0,f,1967
1,0_1390,2021-03-01 00:02:26.047414,s_2,c_664,19.37,0,m,1960
2,0_1352,2021-03-01 00:02:38.311413,s_3,c_580,4.50,0,m,1988
3,0_1458,2021-03-01 00:04:54.559692,s_4,c_7912,6.55,0,f,1989
4,0_1358,2021-03-01 00:05:18.801198,s_5,c_2033,16.49,0,f,1956
...,...,...,...,...,...,...,...,...
687529,1_508,2023-02-28 23:49:03.148402,s_348444,c_3573,21.92,1,f,1996
687530,2_37,2023-02-28 23:51:29.318531,s_348445,c_50,48.99,2,f,1994
687531,1_695,2023-02-28 23:53:18.929676,s_348446,c_488,26.99,1,f,1985
687532,0_1547,2023-02-28 23:58:00.107815,s_348447,c_4848,8.99,0,m,1953


In [39]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 687534 entries, 0 to 687533
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   id_prod     687534 non-null  object 
 1   date        687534 non-null  object 
 2   session_id  687534 non-null  object 
 3   client_id   687534 non-null  object 
 4   price       687534 non-null  float64
 5   categ       687534 non-null  int64  
 6   sex         687534 non-null  object 
 7   birth       687534 non-null  int64  
dtypes: float64(1), int64(2), object(5)
memory usage: 42.0+ MB


In [40]:
df_merge.isna().sum()

id_prod       0
date          0
session_id    0
client_id     0
price         0
categ         0
sex           0
birth         0
dtype: int64

In [41]:
today = datetime.now()

In [42]:
df_merge['date'] = pd.to_datetime(df_merge['date'], format='mixed', errors='coerce')


In [43]:
df_merge.head()

,id_prod,date,session_id,client_id,price,categ,sex,birth
0,0_1259,2021-03-01 00:01:07.843138,s_1,c_329,11.99,0,f,1967
1,0_1390,2021-03-01 00:02:26.047414,s_2,c_664,19.37,0,m,1960
2,0_1352,2021-03-01 00:02:38.311413,s_3,c_580,4.50,0,m,1988
3,0_1458,2021-03-01 00:04:54.559692,s_4,c_7912,6.55,0,f,1989
4,0_1358,2021-03-01 00:05:18.801198,s_5,c_2033,16.49,0,f,1956


In [44]:
df_merge.describe()

,date,price,categ,birth
count,687298,687534.000000,687534.000000,687534.000000
mean,2022-03-01 22:34:32.406761728,17.493918,0.448789,1977.817391
min,2021-03-01 00:01:07.843138,0.620000,0.000000,1929.000000
25%,2021-09-10 09:23:21.452309760,8.990000,0.000000,1970.000000
50%,2022-02-27 09:25:30.888587008,13.990000,0.000000,1980.000000
75%,2022-08-28 23:26:51.933032448,19.080000,1.000000,1987.000000
max,2023-02-28 23:58:30.792755,300.000000,2.000000,2004.000000
std,NaN,18.238337,0.594563,13.607935


In [45]:
rfm = df_merge.groupby('client_id').agg(
    {
        'date' : lambda x: (today - x.max()).days, # Calcul du nombre de jours entre la date d'aujourd'hui et la date de la dernière commande par clients
        'client_id' : 'count',
        'price' : 'sum',
        }
    ).rename(columns={
            'date' : 'Recency',
            'client_id' : 'Frequency',
            'price' : 'Monetary',
        }
    ).reset_index()

In [46]:
rfm

,client_id,Recency,Frequency,Monetary
0,c_1,499,43,629.02
1,c_10,488,58,1353.60
2,c_100,617,8,254.85
3,c_1000,490,126,2291.88
4,c_1001,460,103,1823.85
...,...,...,...,...
8595,c_995,554,14,189.41
8596,c_996,463,96,1637.34
8597,c_997,479,59,1490.01
8598,c_998,461,55,2822.22
